# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## Simple Assembly Line and Balancing Problem (SALBP)

Assembly lines are standard manufacturing systems present in large-scale production. Given a scenario of labour high costs, industries try to implement efficient procedures to plan their activities in order to be competitive against developing countries.

Line balancing is a classical combinatorial optimization problem introduced by Salveson (1955). We focus on its most studied variant: the *Simple Assembly Line Balancing Problem* (SALBP). We consider a set of tasks $N=\{1,...,n\}$, and an ordered set of stations $S=\{1,...,m\}$, $n,m \in \mathbb{N}$. Each task $i \in N$ has an execution time defined by $p_i \in \mathbb{R}_+$. Let $s(i) \in S$ be a station where task $i$ is assigned. We model technological characteristics of the line by precedence constraints between tasks such that $i \preceq j$ ($i$ immediate precedes $j$) means that $s(i) \le s(j)$. Consider $P_i \subseteq N$ as a set of immediate predecessors of task $i$ and $F_i \subseteq N$ as a set of immediate followers of task $i$. We define $c \in \mathbb{R}_+$ as the cycle time of the line. 

The feasibility version of the problem, called SALBP-F, consists in answering the following question: **"Is there a task assignment to stations that respects precedence and cycle time constraints, given a pre-defined number of stations $m$ and a cycle time $c$?"**. The SALBP-F is a NP-Hard problem, since it reduces to a *Partitioning Problem* (Scholl, 1999; Wee & Magazine, 1982). Figure shown below (Moreira, 2015) illustrates the SALBP-F through an example with $c=8$ and $m=5$.

![](images/salbp-f.png)

### SALBP-1

The *Simple Assembly Line Balancing Problem type 1* (SALBP-1) minimizes the number of opened stations, given a cycle time $\overline{c}$. The SALBP-1 appears in situations where we can have a good estimate of the demand, allowing a fixation of cycle time *a priori* and minimizing installation costs by means of number of stations. As follows, we present a feasible solution for this problem (Moreira, 2015), using the previous instance with $\overline{c}=9$ and $m=4$.

![(Moreira, 2015)](images/salbp-1.png)

#### Mathematical model  

The mathematical model shown in this section is based on Patterson & Albracht (1975) formulation. We define $n+1$ as a ficticious task such that $P_{n+1} = \{i \in N; F_i = \varnothing\}$ and $t_{n+1} = 0$. The objective function minimizes the station index where task $q$ is assigned, as a way to minimize the number of stations. 

* **Importing libraries**

In [1]:
# Importing libraries
using JuMP
using Cbc
include("codes/instanceALs.jl")
; # Disable output messages after the block

* **Reading instance**

In [2]:
# SALBP instance
fileInstance="instances/salbp/instance_n=20_1.alb"

io=open(fileInstance)

# Reading SALBP instance
instance = readSALBP(io)
;

* **Creating model**

In [112]:
model = Model(with_optimizer(Cbc.Optimizer))
;

* **Sets and constants (inputs)**

In [113]:
n = instance.n # number of tasks
m = instance.n # number of stations
P = [Vector{Int}() for _ in 1:n+1] # set of immediate predecessors
F = [Vector{Int}() for _ in 1:n+1] # set of immediate followers
c = instance.c #cycle time
p = instance.p

for i=1:n
    P[i] = copy(instance.P[i])
    F[i] = copy(instance.F[i])
    if(size(F[i],1) == 0)
        push!(P[n+1], i)
    end
end

S = collect(1:n+1) # set of stations
N = collect(1:n+1) # Task n+1 corresponds to the artificial task
;

* **Decision variables**

    * $x_{si} \in \{0,1\}$: equal to one iff task $i$ is assigned to station $s$.

In [114]:
@variable(model, x[S,N], Bin)
;

* **Creating objective function**

$\min \sum_{s \in S}s\cdot x_{s,n+1}$

In [115]:
@objective(model, Min, sum(s*x[s,n+1] for s in S))
;

* **Constraint:** each task must be assigned to a single station

$\sum_{s \in S} x_{si} = 1 \qquad \forall i \in N \cup \{n+1\}$

In [116]:
@constraint(model, [i in N], sum(x[s,i] for s in S) == 1)
;

* **Constraint:** precedence constraints must be respected

$\sum_{s \in S}s\cdot x_{si} \le \sum_{s \in S}s\cdot x_{sj} \qquad \forall j \in N \cup \{n+1\}, \forall i \in P_j$

In [117]:
@constraint(model, [j in N, i in P[j]], sum(s*x[s,i] for s in S) <= sum(s*x[s,j] for s in S))
;

* **Constraint:** definition of cycle time as the amount of time of the most loaded station

$$\sum_{i \in N} p_ix_{si} \le \overline{c} \qquad \forall s \in S$$

In [118]:
@constraint(model, [s in S], sum(p[i]*x[s,i] for i in N if i != n+1) <= c) # We eliminate n+1 because p_{n+1} = 0
;

* **Running the model**

In [119]:
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 1.95906 - 0.00 seconds
Cgl0005I 21 SOS with 441 members
Cgl0004I processed model has 65 rows, 441 columns (441 integer (441 of which binary)) and 1827 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 43 integers unsatisfied sum - 14.0409
Cbc0038I Pass   1: suminf.    2.88563 (29) obj. 3 iterations 49
Cbc0038I Pass   2: suminf.    2.74813 (27) obj. 3 iterations 4
Cbc0038I Pass   3: suminf.    2.93041 (22) obj. 20 iterations 17
Cbc0038I Pass   4: suminf.    2.73729 (22) obj. 21 iterations 6
Cbc0038I Pass   5: suminf.    2.34351 (10) obj. 21 iterations 25
Cbc0038I Pass   6: suminf.    2.34351 (10) obj. 21 iterations 0
Cbc0038I Pass   7: suminf.    1.70526 (8) obj. 21 iterations 5
Cbc0038I Pass   8: suminf.    2.00000 (8) obj. 21 iterations 5
Cbc0038I Pass   9: suminf.    1.80000 (8) obj. 21

* **Printing variables**

In [120]:
println("Printing solution")
sOpt = JuMP.objective_value(model) # Optimal solution
for s=1:sOpt
    println("Station ", Int(s), ": ")
    tmp = 0 # Overload of each station
    for i=1:n+1
        if(value(x[s,i]) >= 0.9 && i != n+1)
            print(i, " ")
            if(i != n+1)
                tmp += p[i]
            end
        end
    end
    println("(", tmp, ")")
end
    

Printing solution
Station 1: 
1 2 6 7 10 11 13 17 (957)
Station 2: 
4 8 12 14 18 (991)
Station 3: 
3 5 9 15 16 19 20 (934)


### SALBP-2

The *Simple Assembly Line Balancing Problem type 2* (SALBP-2) minimizes the cycle time, given a number of stations $m$. The SALBP-2 appears in situations where we have already the line installed and we want to increase the efficiency of the production. As follows, we present a feasible solution for this problem (Moreira, 2015), using the previous instance with $c=18$ and $m=3$.

![(Moreira, 2015)](images/salbp-2.png)

#### Mathematical model  

The mathematical model shown in this section uses binary variables $x_{si}$. We define $c \in \mathbb{R}_+$ as a variable that measures the cycle time. The mathematical model of SALBP-2 reads as follows.

\begin{equation}
\min c
\end{equation}

subject to

\begin{alignat}{2}
\sum_{s \in S} x_{sj} = 1 &&  \qquad & \forall j \in N\\
\sum_{s \in S} sx_{si} \le \sum_{s \in S} sx_{sj} &&  \qquad &\forall j \in N, \forall i \in P_j\\
\sum_{j \in N} p_jx_{sj} \le c &&  \qquad & \forall s \in S\\
x_{sj} \in \{0,1\} && \quad &\forall s \in S, \forall j \in N.
\end{alignat}

* **Creating model**

In [128]:
# "logLevel = 0" silences the solver
model = Model(with_optimizer(Cbc.Optimizer, logLevel = 0)) 
;

* **Sets and constants (inputs)**

In [129]:
P = [Vector{Int}() for _ in 1:n] # set of immediate predecessors
F = [Vector{Int}() for _ in 1:n] # set of immediate followers

# Dictionary of upper bounds for SALBP-2
ubS = Dict(
    "instance_n=20_1.alb" =>  3,
    "instance_n=20_2.alb" => 3,
    "instance_n=20_3.alb" => 3,
    "instance_n=50_1.alb" => 8,
    "instance_n=50_2.alb" => 6,
    "instance_n=50_3.alb" => 8,
    "instance_n=100_1.alb" => 23
)

for i=1:n
    P[i] = copy(instance.P[i])
    F[i] = copy(instance.F[i])
end

# Getting instance name
subStrIdx = findfirst("instance_n=",fileInstance)
subStr = fileInstance[subStrIdx[1]:end]

# Setting the upper bound of number of stations
m = ubS[subStr]

S = collect(1:m) # set of stations
N = collect(1:n) # set of tasks
;

* **Decision variables**

    * $x_{si} \in \{0,1\}$: equal to one iff task $i$ is assigned to station $s$.  
    * $c \in \mathbb{R}_+$: cycle time

In [130]:
@variable(model, x[S,N], Bin)
@variable(model, c)
;

* **Creating objective function**

$\min c$

In [131]:
@objective(model, Min, c)
;

* **Constraints**

In [132]:
@constraints(model,
    begin
        [i in N], sum(x[s,i] for s in S) == 1
        [j in N, i in P[j]], sum(s*x[s,i] for s in S) <= sum(s*x[s,j] for s in S)
        [s in S], sum(p[i]*x[s,i] for i in N) <= c
    end)
;

* **Running the model**

In [133]:
optimize!(model)

* **Printing variables**

In [134]:
println("Printing solution")
cOpt = JuMP.objective_value(model) # Optimal solution
println("Cycle time = ", cOpt)
for s=1:m
    println("Station ", Int(s), ": ")
    tmp = 0 # Overload of each station
    for i=1:n
        if(value(x[s,i]) >= 0.9 && i != n+1)
            print(i, " ")
            tmp += p[i]
        end
    end
    println("(", tmp, ")")
end

Printing solution
Cycle time = 962.0
Station 1: 
1 2 3 4 7 8 11 (959)
Station 2: 
6 10 13 16 18 (961)
Station 3: 
5 9 12 14 15 17 19 20 (962)


## References

Moreira, M. C. O. (2015). Problema de balanceamento de linhas de produção e integração de trabalhadores. Tese (Doutorado em Ciências de Computação e Matemática Computacional) - Instituto de Ciências Matemáticas e de Computação, Universidade de São Paulo, São Carlos. Disponível em: http://www.teses.usp.br/teses/disponiveis/55/55134/tde-08012016-145627/pt-br.php

Patterson, J. H., & Albracht, J. J. (1975). Assembly-line balancing: zero-one programming with Fibonacci search. Operations Research, 23(1), 166-172.

Salveson, M. (1955). The assembly line balancing problem. The Journal of Industrial Engineering, 3:18-25.

Scholl (1999) A. Scholl. Balancing and sequencing of assembly lines. Physica-Verlag.

Wee, T. S., & Magazine, M. J. (1982). Assembly line balancing as generalized bin packing. Operations Research Letters, 1(2), 56-58.